# Usage demo for NegativeBinomialLikelihoodVariableSelector

In [1]:
import numpy as np
import pandas as pd
from millipede import NegativeBinomialLikelihoodVariableSelector

## First we create a demo dataset with 3 causal and 97 spurious features

In [2]:
num_datapoints = 250
num_covariates = 100

# create covariates
X = np.random.RandomState(0).randn(num_datapoints * num_covariates)
X = X.reshape((num_datapoints, num_covariates))

# specify the true causal coefficients
true_coefficients = np.array([1.0, -0.5, 0.25] + [0.0] * 97)
print("true_coefficients:\n", true_coefficients)

true_coefficients:
 [ 1.   -0.5   0.25  0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.  ]


In [3]:
# compute responses using a noisy (generalized) linear model
true_intercept = 1.23
poisson_log_rate = X @ true_coefficients + true_intercept
# we add additional noise (and thus additional dispersion in the responses)
poisson_log_rate += 0.3 * np.random.RandomState(1).randn(num_datapoints)
poisson_rate = np.exp(poisson_log_rate)
Y = np.random.RandomState(2).poisson(poisson_rate)
print("Observed counts Y:\n", Y)

# put the covariates, responses, and psi0 vector into a single numpy array
psi0 = np.zeros(num_datapoints)
Ypsi0X = np.concatenate([Y[:, None], psi0[:, None], X], axis=-1)
print("\nX.shape: ", X.shape, "  Y.shape: ", Y.shape, 
      "  psi0.shape: ", psi0.shape, "  Ypsi0X.shape: ", Ypsi0X.shape)

Observed counts Y:
 [33 27  2  0  4  4  1  1  9  0  3  0  5  2  4  4  4  0 14  3  0  5  3  3
  7  0  5  1  5  1 15  9  1  0  0  2  4  0  1  0  2  0  1  6  3  7 17  0
  4 16  7  4  0 14  1  1 11  1  7  3 35  1  7 11 16  6 32  3  1  1  0  7
  0  1 15  1  9  2  5  2  6 10  8  3 40 17  7  2  4  0 14  3 27  1 34  0
  4  5  1  0  6  7 13 17  3 10  0  8  0  9  2  2 18  3  2  6  1  2  6 23
  4  4 10  0  0 12 11 10  3  0  6  2  2  7  0  2 68  0 20 18  1  2  2 17
  7  5 40  3  1  4 28  0  6 21  5  1 11  6  4  0  2  3  7  1  6 50 12  1
  2  1  6  5 11  1 10  1 19  0  3  0  1  0  0  2  0  2  6  2  7  6 21  1
  4  3  7  0  1  1 27 16  4 21  1 40  0  2  3 36  3  2  3  0 10  2  6  5
  1  2 22  1  2  0 21  3  0 11  9  0  0  0  5  3 12  3  3  0  2  2  7  1
  1 10  5 20  2  1  9  1 65  8]

X.shape:  (250, 100)   Y.shape:  (250,)   psi0.shape:  (250,)   Ypsi0X.shape:  (250, 102)


## Then we package the data as a Pandas DataFrame, giving each covariate a  unique name

In [4]:
columns = ['Response', 'Psi0', 'Causal1', 'Causal2', 'Causal3']
columns += ['Spurious{}'.format(k) for k in range(1, 98)]
dataframe = pd.DataFrame(Ypsi0X, columns=columns)
dataframe.head(5)

,Response,Psi0,Causal1,Causal2,Causal3,Spurious1,Spurious2,Spurious3,Spurious4,Spurious5,...,Spurious88,Spurious89,Spurious90,Spurious91,Spurious92,Spurious93,Spurious94,Spurious95,Spurious96,Spurious97
0,33.0,0.0,1.764052,0.400157,0.978738,2.240893,1.867558,-0.977278,0.950088,-0.151357,...,-0.403177,1.222445,0.208275,0.976639,0.356366,0.706573,0.010500,1.785870,0.126912,0.401989
1,27.0,0.0,1.883151,-1.347759,-1.270485,0.969397,-1.173123,1.943621,-0.413619,-0.747455,...,-1.292857,0.267051,-0.039283,-1.168093,0.523277,-0.171546,0.771791,0.823504,2.163236,1.336528
2,2.0,0.0,-0.369182,-0.239379,1.099660,0.655264,0.640132,-1.616956,-0.024326,-0.738031,...,-0.628088,-0.481027,2.303917,-1.060016,-0.135950,1.136891,0.097725,0.582954,-0.399449,0.370056
3,0.0,0.0,-1.306527,1.658131,-0.118164,-0.680178,0.666383,-0.460720,-1.334258,-1.346718,...,0.567290,-0.222675,-0.353432,-1.616474,-0.291837,-0.761492,0.857924,1.141102,1.466579,0.852552
4,4.0,0.0,-0.598654,-1.115897,0.766663,0.356293,-1.768538,0.355482,0.814520,0.058926,...,-1.029935,-0.349943,1.100284,1.298022,2.696224,-0.073925,-0.658553,-0.514234,-1.018042,-0.077855


## Next we create a VariableSelector object appropriate for our count-valued responses

In [5]:
selector = NegativeBinomialLikelihoodVariableSelector(dataframe,  # pass in the data
                                                      'Response', # indicate the column of responses
                                                      'Psi0',     # indicate psi0
                                                      S=1,        # specify the expected number of covariates to include a priori
                                                     )

## Finally we run the MCMC algorithm to compute posterior inclusion probabilities (PIPs) and other quantities of interest

In [6]:
selector.run(T=2000, T_burnin=1000, verbosity='bar', seed=2)

  0%|          | 0/3000 [00:00<?, ?it/s]

## The results are available in the selector.summary DataFrame

- As expected only the 3 causal covariates have large PIPs. 
- In addition the true coefficients are identified correctly (up to noise).
- Note that the intercept term does not have a corresponding PIP, since it is always included in the model by assumption.

In [7]:
selector.summary

,PIP,Coefficient,Coefficient StdDev,Conditional Coefficient,Conditional Coefficienti StdDev
Causal1,1.000000,1.079945e+00,0.033605,1.079945,0.033605
Causal2,1.000000,-5.433689e-01,0.036830,-0.543369,0.036830
Causal3,0.999222,2.093379e-01,0.034774,0.209766,0.033495
Spurious1,0.000045,7.177131e-07,0.000196,0.016073,0.024538
Spurious2,0.000036,-2.440187e-07,0.000202,-0.005844,0.030640
...,...,...,...,...,...
Spurious94,0.000037,4.787561e-08,0.000215,0.001659,0.040079
Spurious95,0.000043,-1.791531e-06,0.000281,-0.041918,0.009262
Spurious96,0.000044,-2.187868e-06,0.000340,-0.031647,0.025892
Spurious97,0.000058,-2.010359e-06,0.000405,-0.044039,0.040625


For example the largest spurious PIP is given by:

In [8]:
selector.summary.PIP.values[3:-1].max()

0.004228535078251371

Some additional stats about the MCMC run are available in `selector.stats`:

In [9]:
selector.stats

{'Weight quantiles': '5/10/20/50/90/95:  1.92e-16  1.92e-16  1.23e-01  1.51e+01  1.51e+01  1.51e+01',
 'Weight moments': 'mean/std/min/max:  9.38e+00  7.22e+00  6.40e-17  1.51e+01',
 'nu posterior': '18.325 +- 4.752',
 'log(nu) posterior': '2.872 +- 0.275',
 'Elapsed MCMC time': '4.0 seconds',
 'Mean iteration time': '1.336 ms',
 'Number of retained samples': 2000,
 'Number of burn-in samples': 1000,
 'Adapted xi value': '2.673',
 'Polya-Gamma MH stats': 'Mean acc. prob.: 0.813  Accepted/Attempted: 439/526'}

In particular `selector.stats` contains information about the posterior for dispersion parameter `nu`:

In [10]:
selector.stats['nu posterior']

'18.325 +- 4.752'